<a href="https://colab.research.google.com/github/PurvikaJoshi01/Gemstone-Classification/blob/main/gemstone_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 k

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2 as cv
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import gradio as gr
import PIL
gems_folder = glob('/content/drive/MyDrive/Gemstone_images/train')
print('number of gems we need to classify is {}'.format(len(gems_folder)))

ametrine = '/content/drive/MyDrive/Gemstone_images/train/Ametrine/ametrine_0.jpg'
PIL.Image.open(str(ametrine))

train = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.25,
                                   zoom_range = 0.2,
                                   width_shift_range=0.2,
                                   vertical_flip=True,
                                   horizontal_flip = True)
train = train.flow_from_directory('/content/drive/MyDrive/Gemstone_images/train',
                                                 target_size = (224,224),
                                                 batch_size =20 ,
                                                 class_mode = 'categorical')

gem_labels = [gem for gem in train.class_indices]
gem_labels
val = ImageDataGenerator(rescale = 1./255)
val_data = val.flow_from_directory('/content/drive/MyDrive/Gemstone_images/test',
                                                 target_size = (224, 224),
                                                 batch_size = 20,
                                                 class_mode = 'categorical')
vgg16_model = VGG16(input_shape=[224,224,3], weights='imagenet', include_top=False)
for layer in vgg16_model.layers:
    layer.trainable = False
output = Dense(7, activation='softmax')(Flatten()(vgg16_model.output))
model = Model(inputs=vgg16_model.input, outputs=output)

model.summary()
model.compile(optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy']
)
history = model.fit(
  train,
  validation_data=val_data,
  epochs=20,
  steps_per_epoch=len(train)//20,
  validation_steps=len(val_data)//20
)

vgg19_model = VGG19(input_shape=[224,224,3], weights='imagenet', include_top=False)
for layer in vgg19_model.layers:
    layer.trainable = False
output = Dense(7, activation='softmax')(Flatten()(vgg19_model.output))
model = Model(inputs=vgg19_model.input, outputs=output)
model.compile(optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

history = model.fit(
  train,
  validation_data=val_data,
  epochs=20,
  steps_per_epoch=len(train)//20,
  validation_steps=3
)
InceptionV3_model = InceptionV3(input_shape=[224,224,3], weights='imagenet', include_top=False)
for layer in InceptionV3_model.layers:
    layer.trainable = False
output = Dense(7, activation='softmax')(Flatten()(InceptionV3_model.output))
model = Model(inputs=InceptionV3_model.input, outputs=output)

model.compile(optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

history = model.fit(
  train,
  validation_data=val_data,
  epochs=20,
  steps_per_epoch=len(train)//20,
  validation_steps=3
)
def gem_indentify(image):
  img=image.reshape(-1,224,224,3)
  pred=model.predict(img)[0]
  return {gem_labels[i]: float(pred[i]) for i in range(7)}

image = gr.inputs.Image(shape=(224,224))
label = gr.outputs.Label(num_top_classes=87)
gr.Interface(fn=gem_indentify, inputs=image, outputs=label,interpretation='default').launch()


number of gems we need to classify is 1
Found 2436 images belonging to 87 classes.
Found 348 images belonging to 87 classes.
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    


InvalidArgumentError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gradio as gr

# Define the image size and number of classes
img_size = (224, 224)
num_classes = 87

# Define the train and validation data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.25,
    zoom_range=0.2,
    width_shift_range=0.2,
    vertical_flip=True,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Gemstone_images/train',
    target_size=img_size,
    batch_size=20,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Gemstone_images/test',
    target_size=img_size,
    batch_size=20,
    class_mode='categorical'
)

# Load the pre-trained VGG16 model
base_model = VGG16(input_shape=(img_size[0], img_size[1], 3), weights='imagenet', include_top=False)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the pre-trained model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# Define the function for gem identification
def gem_identify(image):
    img = image.reshape(-1, img_size[0], img_size[1], 3)
    pred = model.predict(img)[0]
    return {train_generator.class_indices[i]: float(pred[i]) for i in range(num_classes)}

# Create the Gradio interface
image_input = gr.inputs.Image(shape=(img_size[0], img_size[1]))
label_output = gr.outputs.Label(num_top_classes=num_classes)

gr.Interface(fn=gem_identify, inputs=image_input, outputs=label_output, interpretation='default').launch()


Found 2436 images belonging to 87 classes.
Found 348 images belonging to 87 classes.
Epoch 1/20
122/122 [==============================] - 1696s 14s/step - loss: 4.5112 - accuracy: 0.0435 - val_loss: 4.1620 - val_accuracy: 0.0575
Epoch 2/20
122/122 [==============================] - 1638s 13s/step - loss: 3.8352 - accuracy: 0.0936 - val_loss: 3.4922 - val_accuracy: 0.1236
Epoch 3/20
122/122 [==============================] - 1626s 13s/step - loss: 3.1401 - accuracy: 0.1942 - val_loss: 2.8544 - val_accuracy: 0.2644
Epoch 4/20
122/122 [==============================] - 1628s 13s/step - loss: 2.6161 - accuracy: 0.2923 - val_loss: 2.5685 - val_accuracy: 0.3161
Epoch 5/20
122/122 [==============================] - 1696s 14s/step - loss: 2.3236 - accuracy: 0.3584 - val_loss: 2.4507 - val_accuracy: 0.3305
Epoch 6/20
122/122 [==============================] - 1718s 14s/step - loss: 2.0807 - accuracy: 0.4163 - val_loss: 2.1860 - val_accuracy: 0.3937
Epoch 7/20
122/122 [=========================